# CNN实现Mnist手写数字识别

In [14]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [15]:
#读取数据
mnist_data = input_data.read_data_sets('./MNIST_data/',one_hot=True)

KeyboardInterrupt: 

In [4]:
#占位符
x = tf.placeholder(tf.float32,[None,784],name = 'x')
y = tf.placeholder(tf.float32,[None,10],name = 'y')

#用卷积的话需要把784变成28*28
x_image = tf.reshape(x,[-1,28,28,1])

In [5]:
#定义卷积层
def get_weights(shape):
    return tf.Variable(tf.truncated_normal(shape,stddev=0.1))

def get_bias(shape):
    return tf.Variable(tf.constant(0.1,shape=shape))

def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

In [6]:
#第一层计算
w_conv1 = get_weights([5,5,1,32])
b_conv1 = get_bias([32])
h_conv1 = tf.nn.relu(conv2d(x_image,w_conv1)+b_conv1)
o_conv1 = max_pool_2x2(h_conv1) 
# print(o_conv1.shape)

In [7]:
#第二层计算
w_conv2 = get_weights([5,5,32,64])
b_conv2 = get_bias([64])
h_conv2 = tf.nn.relu(conv2d(o_conv1,w_conv2)+b_conv2)
o_conv2 = max_pool_2x2(h_conv2) 
#print(o_conv2.shape)

In [8]:
#两层卷积层之后是全连接层
#第一层全连接
w_fc1 = get_weights([7*7*64,1024])
b_fc1 = get_bias([1024])
o_conv2_flattern = tf.reshape(o_conv2,[-1,7*7*64])
o_fc1 = tf.nn.relu(tf.matmul(o_conv2_flattern,w_fc1)+b_fc1)
keep_prob = tf.placeholder(tf.float32)#随机失活
h_fc1 = tf.nn.dropout(o_fc1,keep_prob=keep_prob)

In [9]:
#第二层全连接
w_fc2 = get_weights([1024,10])
b_fc2 = get_bias([10])
y_ = tf.matmul(h_fc1,w_fc2)+b_fc2

In [10]:
#定义交叉熵损失
loss = tf.nn.softmax_cross_entropy_with_logits_v2(logits=y_,labels=y)
train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))
/Users/caijie/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullar

In [ ]:
#测试准确性
correct_prediction = tf.equal(tf.argmax(y_,1),tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
#定义会话 训练过程
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_step = 20000
    for i in range(train_step):
        batch = mnist_data.train.next_batch(50)
        _,loss_ = sess.run([train_step,loss,accuracy],feed_dict={x:batch[0],y:batch[1],keep_prob=0.5})
        print('loss:%f' % loss_)
        if i%100 == 0:
            accuracy_ = sess.run([train_step,loss,accuracy],feed_dict={x:batch[0],y:batch[1],keep_prob=1.0})
            print('acc:%f' % accuracy_)
    print('test_accuracy: %g' % accuracy.eval(feed_dict={x:mnist_data.test.images,y:mnist_data.test.labels,keep_prob=1.0}))